In [10]:
import os

import albumentations as A
import rich.progress as rprogress
import torch
import torchmetrics.functional
from rich.traceback import install

from types import SimpleNamespace

from src.callbacks import ModelCheckpoint
from src.datasets import MultiDomainDataset, DomainRole, PreprocessingPipeline
from src.models import MLDG, BaseLearner, Encoder, Classifier, compute_metrics, ERM
from rich import print


install(show_locals=False)

args = {
    "num_classes": 7,
    "batch_size": 8,
    "nonlinear_classifier": False,
    "dropout": 0.,
    "lr": 1e-5,
    "weight_decay": 0.,
    "beta": 1.0,
}
args = SimpleNamespace(**args)


# Backend: albumentations
transform = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(
        mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
    ),
    A.ToTensorV2(),
])

augment_transform = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3, p=0.5),
    A.ToGray(num_output_channels=3, p=0.10),
    A.Normalize(
        mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
    ),
    A.ToTensorV2(),
])


def wrapper_albumentations_transform(transform: A.Compose):
    def transform_(img):
        img_ = img.permute(1, 2, 0).numpy()
        try:
            return transform(image=img_).get("image")
        except Exception as error:
            print(f"Failed to apply transform at image {img_.shape}, {img_.dtype}")
            raise error

    return transform_

pipeline = PreprocessingPipeline(
    source_transform=wrapper_albumentations_transform(augment_transform),
    target_transform=wrapper_albumentations_transform(transform),
)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device selected: {device}")

Device selected: cuda

In [3]:
domains = ["art_painting", "cartoon", "photo", "sketch"]
roles = [DomainRole.TARGET, DomainRole.TARGET, DomainRole.TARGET, DomainRole.TARGET]


In [11]:
def load_model(checkpoint, args):

    if args.baseline:
        model = ERM(
        network=BaseLearner(
            encoder=Encoder(hparams={}),
            classifier=Classifier(hparams={"num_feats": 2048, "num_classes": args.num_classes, "dropout": args.dropout})
        ),
        device=device,
        num_classes=args.num_classes,
        num_domains=len([role for role in roles if role == DomainRole.TARGET]),
        hparams={
            "lr": args.lr,
            "weight_decay": args.weight_decay,
        }
    )

    else:
        model = MLDG(
        network=BaseLearner(
            encoder=Encoder(hparams={}),
            classifier=Classifier(hparams={"num_feats": 2048, "num_classes": args.num_classes, "dropout": args.dropout})
        ),
        device=device,
        num_classes=args.num_classes,
        num_domains=len([role for role in roles if role == DomainRole.TARGET]),
        hparams={
            "num_meta_test": 1,
            "lr": args.lr,
            "weight_decay": args.weight_decay,
            "beta": args.beta,
            "lr_clone": args.lr,  # TODO: for now, using same value for both
            "weight_decay_clone": args.weight_decay,
        }
    )

    # Test
    model.load_state_dict(
        state_dict=torch.load(
            checkpoint
        )
    )
    model.eval()
    return model


In [5]:
dataset = MultiDomainDataset(
    root="../data/PACS/",
    domains=domains,
    roles=roles,
    transforms=[pipeline] * len(domains),
    seed=0,
    split_ratio=0.80,
)


progress_bar = rprogress.Progress(
    rprogress.SpinnerColumn(),
    *rprogress.Progress.get_default_columns(),
    rprogress.TimeElapsedColumn(),
    rprogress.TextColumn("{task.fields[metrics]}", justify="right"),
)

loaders = dataset.build_target_dataloaders(
    use_splits=False, batch_size=args.batch_size, num_workers=8,
    shuffle=False, persistent_workers=True
)

In [19]:
RESULTS = {
    "CHECKPOINTS": [
        "../dump/dutiful-leaf-69_val.ckpt", # photo
        "../dump/polar-mountain-68_val.ckpt",  # art
        "../dump/hearty-microwave-67_val.ckpt",  # cartoon
        "../dump/vague-armadillo-66_val.ckpt",  # sketch
    ],
    "TEST_ACC": [],
    "TEST_MACRO": [],
    "TEST_ACC_PER_DOMAIN": [],
    "TEST_ACC_PER_CLASS": [],
}

In [20]:
for checkpoint in RESULTS["CHECKPOINTS"]:
    if not os.path.exists(checkpoint):
        print(f"Failed to locate checkpoint {checkpoint}")

In [21]:
args

namespace(num_classes=7,
          batch_size=8,
          nonlinear_classifier=False,
          dropout=0.0,
          lr=1e-05,
          weight_decay=0.0,
          beta=1.0)

In [22]:
import torch.nn.functional as F

def compute_metrics_no_agg(model, batch):
    acc = torch.zeros(model.num_classes).to(model.device)
    count = 0
    assert len(batch) == 1
    for (_, x, y) in batch:
        x, y = x.to(model.device), y.to(model.device)
        logits = model.network(x)
        probs = F.softmax(logits, dim=1)
        labels = torch.argmax(probs, dim=1)
        acc_ = torchmetrics.functional.accuracy(labels, y, task="multiclass", num_classes=model.num_classes, average="none")
        acc += acc_
        count += 1
    acc /= count
    return acc


def compute_metrics_macro(model, batch):
    acc = 0
    count = 0
    assert len(batch) == 1
    for (_, x, y) in batch:
        x, y = x.to(model.device), y.to(model.device)
        logits = model.network(x)
        probs = F.softmax(logits, dim=1)
        labels = torch.argmax(probs, dim=1)
        acc_ = torchmetrics.functional.accuracy(labels, y, task="multiclass", num_classes=model.num_classes, average="macro")
        acc += acc_
        count += 1
    acc /= count
    return acc

In [ ]:
with progress_bar:
    for i, CHECKPOINT in enumerate(RESULTS["CHECKPOINTS"]):
        model = load_model(CHECKPOINT, args)


        tasks = [
            progress_bar.add_task(
                description=f"Testing {dataset.domains[i]}",
                total=len(loader),
                metrics=f"acc: --",
            )
            for i, loader in enumerate(loaders)
        ]


        ACC = []
        ACC_MACRO = []
        ACC_PER_DOMAIN = []
        ACC_PER_CLASS = []
        for task, loader in zip(tasks, loaders):
            acc = 0
            acc_macro = 0
            acc_per_class = torch.zeros(model.num_classes)
            count = 0
            for batch in loader:
                acc_ = model.log_metrics((batch,))
                acc_macro_ = compute_metrics_macro(model, (batch, ))
                acc_per_class_ = compute_metrics_no_agg(model, (batch, ))

                # print(f"acc_ {acc_.shape}")
                # print(f"acc_per_class_ {acc_per_class_.shape}")

                count += 1
                acc += acc_
                acc_macro += acc_macro_
                acc_per_class += acc_per_class_.cpu().numpy()

                progress_bar.update(
                    task,
                    advance=1,
                    metrics=f"acc: {acc_:.2%}",
                )

            acc /= count
            acc_macro /= count
            acc_per_class /= count

            progress_bar.update(
                task,
                metrics=f"acc: {acc:.2%}",
            )
            ACC.append(acc)
            ACC_MACRO.append(acc_macro)
            ACC_PER_CLASS.append(acc_per_class)

        RESULTS["TEST_ACC"].append(ACC[i])
        RESULTS["TEST_MACRO"].append(ACC_MACRO[i])
        RESULTS["TEST_ACC_PER_DOMAIN"].append(ACC)
        RESULTS["TEST_ACC_PER_CLASS"].append(ACC_PER_CLASS)

In [24]:
dataset.domains

['art_painting', 'cartoon', 'photo', 'sketch']

In [14]:
acc_per_class

tensor([0.1156, 0.1105, 0.1646, 0.1458, 0.1044, 0.0122, 0.0077])

In [16]:
print(RESULTS)

{
    'CHECKPOINTS': [
        '../dump/dutiful-leaf-69_val.ckpt',
        '../dump/polar-mountain-68_val.ckpt',
        '../dump/hearty-microwave-67_val.ckpt',
        '../dump/vague-armadillo-66_val.ckpt'
    ],
    'TEST_ACC': [
        tensor(0.7393, device='cuda:0'),
        tensor(0.6779, device='cuda:0'),
        tensor(0.9486, device='cuda:0'),
        tensor(0.6550, device='cuda:0')
    ],
    'TEST_MACRO': [
        tensor(0.4354, device='cuda:0'),
        tensor(0.3909, device='cuda:0'),
        tensor(0.8237, device='cuda:0'),
        tensor(0.3478, device='cuda:0')
    ],
    'TEST_ACC_PER_DOMAIN': [
        [
            tensor(0.7393, device='cuda:0'),
            tensor(0.8584, device='cuda:0'),
            tensor(0.9821, device='cuda:0'),
            tensor(0.8740, device='cuda:0')
        ],
        [
            tensor(0.9121, device='cuda:0'),
            tensor(0.6779, device='cuda:0'),
            tensor(0.9868, device='cuda:0'),
            tensor(0.8608, device='cuda:0')
        ],
        [
            tensor(0.8511, device='cuda:0'),
            tensor(0.8959, device='cuda:0'),
            tensor(0.9486, device='cuda:0'),
            tensor(0.8910, device='cuda:0')
        ],
        [
            tensor(0.9150, device='cuda:0'),
            tensor(0.8409, device='cuda:0'),
            tensor(0.9916, device='cuda:0'),
            tensor(0.6550, device='cuda:0')
        ]
    ],
    'TEST_ACC_PER_CLASS': [
        [
            tensor([0.1427, 0.0829, 0.1328, 0.0422, 0.0713, 0.1426, 0.1363]),
            tensor([0.1591, 0.1730, 0.1455, 0.0614, 0.0922, 0.1186, 0.1239]),
            tensor([0.1112, 0.1280, 0.1112, 0.1106, 0.1141, 0.1675, 0.2632]),
            tensor([0.1606, 0.1664, 0.1498, 0.1547, 0.1939, 0.0196, 0.0368])
        ],
        [
            tensor([0.1748, 0.1206, 0.1367, 0.0815, 0.0864, 0.1426, 0.1880]),
            tensor([0.1201, 0.1247, 0.1328, 0.0614, 0.0499, 0.1126, 0.0869]),
            tensor([0.1130, 0.1292, 0.1118, 0.1142, 0.1118, 0.1675, 0.2632]),
            tensor([0.1480, 0.1730, 0.1809, 0.1562, 0.1748, 0.0112, 0.0231])
        ],
        [
            tensor([0.1593, 0.1172, 0.1265, 0.0648, 0.0874, 0.1436, 0.1667]),
            tensor([0.1553, 0.1840, 0.1468, 0.0614, 0.0994, 0.1186, 0.1460]),
            tensor([0.1077, 0.1214, 0.0965, 0.1082, 0.1076, 0.1675, 0.2632]),
            tensor([0.1682, 0.1702, 0.1627, 0.1560, 0.1865, 0.0183, 0.0367])
        ],
        [
            tensor([0.1738, 0.1187, 0.1401, 0.0824, 0.0898, 0.1436, 0.1857]),
            tensor([0.1350, 0.1691, 0.1480, 0.0606, 0.1045, 0.1207, 0.1179]),
            tensor([0.1142, 0.1292, 0.1136, 0.1130, 0.1147, 0.1675, 0.2632]),
            tensor([0.1156, 0.1105, 0.1646, 0.1458, 0.1044, 0.0122, 0.0077])
        ]
    ]
}

In [19]:
dlist = dataset.domains

for domain, acc in zip(dlist, RESULTS["TEST_ACC"]):
    print(f"[bold green]{domain}[/bold green]: {acc:.2%}")
print(f"[bold green]Average[/bold green]: {torch.mean(torch.tensor(RESULTS["TEST_ACC"])).item():.2%}")

for domain, acc in zip(dlist, RESULTS["TEST_MACRO"]):
    print(f"[bold green]{domain}[/bold green]: {acc:.2%}")

for domain, acc in zip(dlist, RESULTS["TEST_ACC_PER_DOMAIN"]):
    print(f"[bold green]{domain}[/bold green]: {[f'{a:.2%}' for a in acc]}")

cls_to_idx = dataset.datasets[0].class_to_idx.copy()
idx_to_cls = {v: k for k, v in cls_to_idx.items()}
names = list(map(idx_to_cls.get, range(7)))
print(names)
for j, acc in enumerate(RESULTS["TEST_ACC_PER_CLASS"]):
    vals = {k: f"{v.item():.2%}" for i, (k, v) in enumerate(zip(names, acc[j]))}
    print(f"[bold green]{domains[j]}[/bold green]: {vals}")


art_painting: 73.93%

cartoon: 67.79%

photo: 94.86%

sketch: 65.50%

Average: 75.52%

...

['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']

art_painting: {'dog': '14.27%', 'elephant': '8.29%', 'giraffe': '13.28%', 'guitar': '4.22%', 'horse': '7.13%', 
'house': '14.26%', 'person': '13.63%'}

cartoon: {'dog': '12.01%', 'elephant': '12.47%', 'giraffe': '13.28%', 'guitar': '6.14%', 'horse': '4.99%', 'house':
'11.26%', 'person': '8.69%'}

photo: {'dog': '10.77%', 'elephant': '12.14%', 'giraffe': '9.65%', 'guitar': '10.82%', 'horse': '10.76%', 'house': 
'16.75%', 'person': '26.32%'}

sketch: {'dog': '11.56%', 'elephant': '11.05%', 'giraffe': '16.46%', 'guitar': '14.58%', 'horse': '10.44%', 
'house': '1.22%', 'person': '0.77%'}

In [12]:
import torch
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall
)

# Parameters
num_classes = 7
domains = dataset.domains
print(f"Domains: {domains}")

# Helper
domain_to_idx = {domain.lower(): i for i, domain in enumerate(domains)}

# MLDG MODEL
# RESULTS = {
#     "CHECKPOINTS": [
#         "../dump/wobbly-disco-103_val.ckpt",  # sketch
#         "../dump/lively-violet-104_val.ckpt",  # photo
#         "../dump/breezy-pond-105_val.ckpt",  # cartoon
#         "../dump/deep-wave-107_val.ckpt",  # art
#     ],
#     "TEST_DOMAIN": [  # to make sure we dont mix up things
#         "Sketch",
#         "Photo",
#         "Cartoon",
#         "Art_Painting"
#     ],
#     "METRIC_SUMMARY": [],
# }


# BASELINE
RESULTS = {
    "CHECKPOINTS": [
        "../dump/devout-rain-112_val.ckpt",  # sketch
        "../dump/wobbly-shape-113_val.ckpt",  # photo
        "../dump/clean-dust-114_val.ckpt",  # cartoon
        "../dump/vital-bird-115_val.ckpt",  # art
    ],
    "TEST_DOMAIN": [  # to make sure we dont mix up things
        "Sketch",
        "Photo",
        "Cartoon",
        "Art_Painting"
    ],
    "METRIC_SUMMARY": [],
}

args.baseline = True

args.nonlinear_classifier = True

print(f"Checkpoints to load: {
    [(check, test) for check, test in zip(RESULTS['CHECKPOINTS'], RESULTS['TEST_DOMAIN'])]
}")
print(args)

Domains: ['art_painting', 'cartoon', 'photo', 'sketch']

Checkpoints to load: [('../dump/devout-rain-112_val.ckpt', 'Sketch'), ('../dump/wobbly-shape-113_val.ckpt', 
'Photo'), ('../dump/clean-dust-114_val.ckpt', 'Cartoon'), ('../dump/vital-bird-115_val.ckpt', 'Art_Painting')]

namespace(num_classes=7, batch_size=8, nonlinear_classifier=True, dropout=0.0, lr=1e-05, weight_decay=0.0, 
beta=1.0, baseline=True)

In [13]:

with progress_bar:
    for i, (CHECKPOINT, DOMAIN) in enumerate(zip(RESULTS["CHECKPOINTS"], RESULTS["TEST_DOMAIN"])):
        model = load_model(CHECKPOINT, args)
        loader = loaders[domain_to_idx[RESULTS["TEST_DOMAIN"][i].lower()]]
        task = progress_bar.add_task(
            description=f"Testing {dataset.domains[i]}",
            total=len(loader),
            metrics="--"
        )


        # Define metric objects (stateful across batches)
        metric_acc = MulticlassAccuracy(num_classes=num_classes, average='micro')
        metric_f1_macro = MulticlassF1Score(num_classes=num_classes, average='macro')
        metric_f1_weighted = MulticlassF1Score(num_classes=num_classes, average='weighted')
        metric_precision = MulticlassPrecision(num_classes=num_classes, average=None)
        metric_recall = MulticlassRecall(num_classes=num_classes, average=None)
        metric_f1_per_class = MulticlassF1Score(num_classes=num_classes, average=None)

        metric_acc = metric_acc.to(device)
        metric_f1_macro = metric_f1_macro.to(device)
        metric_f1_weighted = metric_f1_weighted.to(device)
        metric_precision = metric_precision.to(device)
        metric_recall = metric_recall.to(device)
        metric_f1_per_class = metric_f1_per_class.to(device)


        for batch in loader:
            with torch.inference_mode():
                # Compute stuff
                (_, x, y_true) = batch
                x, y_true = x.to(model.device), y_true.to(model.device)
                logits = model.network(x)
                probs = torch.nn.functional.softmax(logits, dim=1)
                y_pred = torch.argmax(probs, dim=1)

                 # Update metrics
                metric_acc.update(y_pred, y_true)
                metric_f1_macro.update(y_pred, y_true)
                metric_f1_weighted.update(y_pred, y_true)
                metric_precision.update(y_pred, y_true)
                metric_recall.update(y_pred, y_true)
                metric_f1_per_class.update(y_pred, y_true)

            progress_bar.update(
                task,
                advance=1,
                metrics=f"acc: {metric_acc.compute():.4f}",
            )

        # Compute final results
        acc = metric_acc.compute()
        f1_macro = metric_f1_macro.compute()
        f1_weighted = metric_f1_weighted.compute()
        precision_per_class = metric_precision.compute()
        recall_per_class = metric_recall.compute()
        f1_per_class = metric_f1_per_class.compute()

        RESULTS["METRIC_SUMMARY"].append({
            "acc": acc,
            "f1_macro": f1_macro,
            "f1_weighted": f1_weighted,
            "precision_per_class": precision_per_class,
            "recall_per_class": recall_per_class,
            "f1_per_class": f1_per_class,
        })


        # Display results
        print(f"=== {DOMAIN.lower()} ===")
        print(f"Micro Accuracy:     {acc:.4f}")
        print(f"Macro F1 Score:     {f1_macro:.4f}")
        print(f"Weighted F1 Score:  {f1_weighted:.4f}\n")

        for i in range(num_classes):
            print(f"Class {i}: "
                  f"Precision={precision_per_class[i]:.4f}  "
                  f"Recall={recall_per_class[i]:.4f} "
                  f"F1={f1_per_class[i]:.4f}")


=== sketch ===

Micro Accuracy:     0.7160

Macro F1 Score:     0.7200

Weighted F1 Score:  0.7371

Class 0: Precision=0.7373  Recall=0.5997 F1=0.6614

...

Class 2: Precision=0.7962  Recall=0.8772 F1=0.8347

Class 3: Precision=0.8901  Recall=0.8804 F1=0.8852

Class 4: Precision=0.6948  Recall=0.8607 F1=0.7689

Class 5: Precision=0.9553  Recall=0.9424 F1=0.9488

Class 6: Precision=0.9559  Recall=0.7238 F1=0.8238

In [8]:
print(RESULTS)


{
    'CHECKPOINTS': [
        '../dump/wobbly-disco-103_val.ckpt',
        '../dump/lively-violet-104_val.ckpt',
        '../dump/breezy-pond-105_val.ckpt',
        '../dump/deep-wave-107_val.ckpt'
    ],
    'TEST_DOMAIN': ['Sketch', 'Photo', 'Cartoon', 'Art_Painting'],
    'METRIC_SUMMARY': [
        {
            'acc': tensor(0.8285, device='cuda:0'),
            'f1_macro': tensor(0.8470, device='cuda:0'),
            'f1_weighted': tensor(0.8283, device='cuda:0'),
            'precision_per_class': tensor([0.7317, 0.8765, 0.9168, 0.9617, 0.7144, 1.0000, 0.8411],
       device='cuda:0'),
            'recall_per_class': tensor([0.6749, 0.7676, 0.8194, 0.9901, 0.9167, 0.9000, 0.7937],
       device='cuda:0'),
            'f1_per_class': tensor([0.7022, 0.8184, 0.8654, 0.9757, 0.8030, 0.9474, 0.8167],
       device='cuda:0')
        },
        {
            'acc': tensor(0.9754, device='cuda:0'),
            'f1_macro': tensor(0.9714, device='cuda:0'),
            'f1_weighted': tensor(0.9754, device='cuda:0'),
            'precision_per_class': tensor([0.9534, 0.9797, 0.9886, 0.9945, 0.9300, 0.9689, 0.9954],
       device='cuda:0'),
            'recall_per_class': tensor([0.9735, 0.9554, 0.9560, 0.9731, 0.9347, 1.0000, 0.9977],
       device='cuda:0'),
            'f1_per_class': tensor([0.9634, 0.9674, 0.9721, 0.9837, 0.9323, 0.9842, 0.9965],
       device='cuda:0')
        },
        {
            'acc': tensor(0.7491, device='cuda:0'),
            'f1_macro': tensor(0.7663, device='cuda:0'),
            'f1_weighted': tensor(0.7578, device='cuda:0'),
            'precision_per_class': tensor([0.4636, 0.8191, 0.9589, 0.5895, 0.9554, 1.0000, 0.9805],
       device='cuda:0'),
            'recall_per_class': tensor([0.8843, 0.6937, 0.8757, 1.0000, 0.5957, 0.9132, 0.4963],
       device='cuda:0'),
            'f1_per_class': tensor([0.6083, 0.7512, 0.9154, 0.7418, 0.7338, 0.9546, 0.6590],
       device='cuda:0')
        },
        {
            'acc': tensor(0.8359, device='cuda:0'),
            'f1_macro': tensor(0.8316, device='cuda:0'),
            'f1_weighted': tensor(0.8362, device='cuda:0'),
            'precision_per_class': tensor([0.8276, 0.6853, 0.8978, 0.9922, 0.6857, 0.8831, 0.9545],
       device='cuda:0'),
            'recall_per_class': tensor([0.8865, 0.9137, 0.8632, 0.6902, 0.8358, 0.9729, 0.7016],
       device='cuda:0'),
            'f1_per_class': tensor([0.8561, 0.7832, 0.8801, 0.8141, 0.7534, 0.9258, 0.8087],
       device='cuda:0')
        }
    ]
}

In [9]:
import json
with open("SummaryResults.json", "w") as f:
    json.dump(RESULTS, f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 import json                                                                                  │
│   2 with open("SummaryResults.json", "w") as f:                                                  │
│ ❱ 3 │   json.dump(RESULTS, f)                                                                    │
│   4                                                                                              │
│                                                                                                  │
│ /usr/lib/python3.12/json/__init__.py:179 in dump                                                 │
│                                                                                                  │
│   176 │   │   │   default=default, sort_keys=sort_keys, **kw).iterencode(obj)                    │
│   177 │   # could accelerate with writelines in some versions of Python, at                      │
│   178 │   # a debuggability cost                                                                 │
│ ❱ 179 │   for chunk in iterable:                                                                 │
│   180 │   │   fp.write(chunk)                                                                    │
│   181                                                                                            │
│   182                                                                                            │
│                                                                                                  │
│ /usr/lib/python3.12/json/encoder.py:432 in _iterencode                                           │
│                                                                                                  │
│   429 │   │   elif isinstance(o, (list, tuple)):                                                 │
│   430 │   │   │   yield from _iterencode_list(o, _current_indent_level)                          │
│   431 │   │   elif isinstance(o, dict):                                                          │
│ ❱ 432 │   │   │   yield from _iterencode_dict(o, _current_indent_level)                          │
│   433 │   │   else:                                                                              │
│   434 │   │   │   if markers is not None:                                                        │
│   435 │   │   │   │   markerid = id(o)                                                           │
│                                                                                                  │
│ /usr/lib/python3.12/json/encoder.py:406 in _iterencode_dict                                      │
│                                                                                                  │
│   403 │   │   │   │   │   chunks = _iterencode_dict(value, _current_indent_level)                │
│   404 │   │   │   │   else:                                                                      │
│   405 │   │   │   │   │   chunks = _iterencode(value, _current_indent_level)                     │
│ ❱ 406 │   │   │   │   yield from chunks                                                          │
│   407 │   │   if newline_indent is not None:                                                     │
│   408 │   │   │   _current_indent_level -= 1                                                     │
│   409 │   │   │   yield '\n' + _indent * _current_indent_level                                   │
│                                                                                                  │
│ /usr/lib/python3.12/json/encoder.py:326 in _iterencode_list                                      │
│                                                            